# Creation of Intents

#### Loading Data

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
data = pd.read_csv("question_responce.csv").drop(columns='Unnamed: 0')
data = data.drop_duplicates()
data.head(5)

,responce,question
0,@115904 We'll be sure to pass along your kind words! #AATeam,@AmericanAir Erica on the lax team is amazing give her a raise ty
1,@115904 Our apologies for the delay in responding to you. Have you made it to LAX? Let us know if you still need assistance.,@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP
2,"@115905 Aww, that's definitely a future pilot in the making! #HappyHalloween",Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1
3,@115906 We're sorry for your frustration.,"@AmericanAir Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program."
4,@115909 We're glad you got to kick back and enjoy a show while flying! Thanks for your kind words.,"Thank you, @AmericanAir for playing #ThisIsUs and for having great flight attendants on my flight back home!"


#### Preprocessing 

**Steps :**
- All words starting by '@' and '#' are removed
- Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- All stopwords are removed.
- Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
- bags of words are created

In [4]:
import gensim
import nltk
import numpy as np
import json
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

np.random.seed(2018)
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('words')
words = set(nltk.corpus.words.words())

Extra stopwords

In [5]:
extra_words = ['flight','flights','fly','flying','plane','thanks', 'thank','ty','get','please','plz']
stop_words = stopwords.words('english')
stop_words.extend(extra_words)

In [6]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

#### Preprocessing

Remove unecessary items int the text

In [7]:
def question_preprocess(text):
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    text = " ".join(filter(lambda x:x[0]!='#', text.split()))
    text = " ".join(filter(lambda x:x[0:4]!='http', text.split()))
    return text

In [8]:
data['question_clean'] = data['question'].apply(question_preprocess)
data['question_clean']

0                                                                          Erica on the lax team is amazing give her a raise ty
1                                                 Could you have someone on your lax team available to guide me to my gate ASAP
2                                                                              Ben Tennyson and an American Airlines pilot. 🎃 …
3       Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program.
4                                         Thank you, for playing and for having great flight attendants on my flight back home!
                                                                 ...                                                           
1847                                and have nailed in the transatlantic WiFi service. I am able to join my daily scrum onboard
1848                                                                                Average price of tic

In [9]:
data['question_preproc'] = data['question_clean'].apply(gensim.utils.simple_preprocess)
data['question_preproc']

0                                                                             [erica, on, the, lax, team, is, amazing, give, her, raise, ty]
1                                             [could, you, have, someone, on, your, lax, team, available, to, guide, me, to, my, gate, asap]
2                                                                                        [ben, tennyson, and, an, american, airlines, pilot]
3       [right, but, earned, those, also, shouldn, have, to, pay, to, pass, them, to, my, own, spouse, you, need, to, change, your, program]
4                                        [thank, you, for, playing, and, for, having, great, flight, attendants, on, my, flight, back, home]
                                                                        ...                                                                 
1847                               [and, have, nailed, in, the, transatlantic, wifi, service, am, able, to, join, my, daily, scrum, onboard]
1848         

In [10]:
def remove_stopwords(text):
    return [[w for w in simple_preprocess(str(doc)) if w not in stop_words] for doc in text]

In [11]:
data["question_stop"] = remove_stopwords(data['question_preproc'])
data["question_stop"] = data["question_stop"].apply(" ".join)
data["question_stop"]

0                                                   erica lax team amazing give raise
1                                    could someone lax team available guide gate asap
2                                                ben tennyson american airlines pilot
3                               right earned also pay pass spouse need change program
4                                                  playing great attendants back home
                                            ...                                      
1847                  nailed transatlantic wifi service able join daily scrum onboard
1848                                                     average price ticket one way
1849          really annoyed month since damaged bag claim never heard back done told
1850    terrible service wait ages trying call number almost two months gone response
1851                                 charges patrons change every time airport closes
Name: question_stop, Length: 1852, dtype: object

Lemmatize

In [12]:
#lemmatize with POS tag = V
def lemmat(text):
    result = []
    text
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words and len(token) > 2:
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return result


In [13]:
data ["question_lem"] = data["question_stop"].apply(lemmat)

In [14]:
data[["question","question_lem"]]

,question,question_lem
0,@AmericanAir Erica on the lax team is amazing give her a raise ty,"[erica, lax, team, amaze, give, raise]"
1,@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP,"[could, someone, lax, team, available, guide, gate, asap]"
2,Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1,"[ben, tennyson, american, airlines, pilot]"
3,"@AmericanAir Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program.","[right, earn, also, pay, pass, spouse, need, change, program]"
4,"Thank you, @AmericanAir for playing #ThisIsUs and for having great flight attendants on my flight back home!","[play, great, attendants, back, home]"
...,...,...
1847,@AmericanAir and @172 have nailed in the transatlantic WiFi service. I am able to join my @172377 daily scrum onboard,"[nail, transatlantic, wifi, service, able, join, daily, scrum, onboard]"
1848,@AmericanAir Average price of ticket out: $2500 one way.,"[average, price, ticket, one, way]"
1849,@AmericanAir Really annoyed been over a month since my damaged bag claim never heard back! done as told...,"[really, annoy, month, since, damage, bag, claim, never, hear, back, do, tell]"
1850,@AmericanAir terrible service wait ages trying to call that number almost two months gone no response,"[terrible, service, wait, age, try, call, number, almost, two, months, go, response]"


#### Making bigrams and trigrams

In [15]:
bigram = gensim.models.Phrases(data["question_lem"], min_count=3, threshold=10)
bigram_mod = gensim.models.phrases.Phraser(bigram)

#example
for i in range(15,24):
    print(bigram_mod[data["question_lem"][i]])

# we obtain coherent bigrams as customer_service, business_class, etc..

['god', 'go', 'ten', 'minutes']
['give', 'grief', 'things', 'poorly', 'credit', 'guacamole', 'admirals_club', 'outstanding', 'addition']
['aware', 'fit', 'overhead', 'return_trip', 'luggage', 'go', 'pre', 'security', 'gate', 'chck', 'empty', 'overheads', 'flt']
['despite', 'ewr', 'force', 'mia', 'check', 'guitar', 'amp', 'charge', 'gate', 'security', 'flt', 'full']
['fifteen', 'empty', 'business_class', 'seat', 'one', 'person', 'get', 'upgrade']
['still_wait', 'compensate', 'crap', 'put', 'honeymoon', 'almost', 'year', 'ago']
['really', 'hour', 'international', 'seat', 'back', 'entertainment', 'screen', 'mon']
['still', 'rather', 'screen', 'though', 'rather', 'cause', 'neck', 'problems', 'look', 'tablet']
['say', 'attenants', 'bdl', 'ord', 'awesome', 'gate_agents', 'bdl', 'also', 'need', 'shout', 'awesome', 'customer_service', 'treat', 'nonrev', 'well']


In [16]:
def bigrams(text):
    return [bigram_mod[doc] for doc in text]

In [17]:
data["question_lem_bigrams"] = bigrams(data["question_lem"])

In [18]:
data = data[['responce', 'question','question_lem_bigrams']]

#### Building LDA

In [19]:
import gensim.corpora as corpora

In [20]:
id2word = corpora.Dictionary(data["question_lem_bigrams"])
id2word.filter_extremes(no_below=3)
corpus = [id2word.doc2bow(text) for text in data["question_lem_bigrams"]]

In [21]:
lda_model_10 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10)

In [22]:
lda_model_20 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=20)

In [23]:
lda_model_30 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=30)

In [24]:
lda_model_40 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=40)

In [25]:
lda_model_80 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=80)

In [26]:
print("lda_model_10: ",lda_model_10[corpus[0]])
print("lda_model_20: ",lda_model_20[corpus[0]])
print("lda_model_30: ",lda_model_30[corpus[0]])
print("lda_model_40: ",lda_model_40[corpus[0]])
print("lda_model_80: ",lda_model_80[corpus[0]])

lda_model_10:  [(0, 0.020005595), (1, 0.02000417), (2, 0.8199487), (3, 0.020005241), (4, 0.020006534), (5, 0.020008605), (6, 0.0200052), (7, 0.02000525), (8, 0.020005982), (9, 0.020004762)]
lda_model_20:  [(0, 0.010000518), (1, 0.010000518), (2, 0.010000518), (3, 0.010000518), (4, 0.010000518), (5, 0.010000518), (6, 0.010000518), (7, 0.010000518), (8, 0.010000518), (9, 0.010000518), (10, 0.010000518), (11, 0.010000518), (12, 0.31222904), (13, 0.0100005185), (14, 0.010000518), (15, 0.50776166), (16, 0.010000518), (17, 0.010000518), (18, 0.010000518), (19, 0.010000518)]
lda_model_30:  [(4, 0.80666256)]
lda_model_40:  [(10, 0.31332365), (32, 0.49666384)]
lda_model_80:  [(28, 0.8024955)]


our example corpus[0] is allegedly well classified in a topic by ~80%. but sometimes the lda_models hesitates between topics

display the topic's top 10 words

In [27]:
#example with 20 topics
topwords = np.array(lda_model_20.show_topics(num_topics=80, num_words=10))
for i in enumerate(topwords[:,1]):
    print(i)

(0, '0.014*"great" + 0.014*"never" + 0.012*"land" + 0.012*"later" + 0.012*"delay" + 0.010*"know" + 0.010*"would" + 0.010*"schedule" + 0.010*"make" + 0.010*"way"')
(1, '0.024*"delay" + 0.014*"really" + 0.012*"travel" + 0.011*"experience" + 0.010*"hey" + 0.010*"tell" + 0.010*"time" + 0.010*"lose" + 0.009*"service" + 0.009*"make"')
(2, '0.018*"day" + 0.014*"one" + 0.012*"gate" + 0.012*"make" + 0.010*"ticket" + 0.009*"pilot" + 0.009*"delay" + 0.008*"wait" + 0.007*"trip" + 0.007*"want"')
(3, '0.017*"know" + 0.012*"get" + 0.011*"issue" + 0.010*"seat" + 0.010*"ticket" + 0.009*"see" + 0.009*"still" + 0.009*"way" + 0.009*"one" + 0.008*"day"')
(4, '0.021*"seat" + 0.017*"time" + 0.015*"pay" + 0.015*"charge" + 0.015*"check" + 0.014*"use" + 0.011*"back" + 0.010*"way" + 0.010*"let" + 0.010*"work"')
(5, '0.015*"make" + 0.014*"pay" + 0.013*"another" + 0.011*"miami" + 0.011*"delay" + 0.010*"check" + 0.010*"get" + 0.010*"would" + 0.010*"stop" + 0.009*"gate"')
(6, '0.016*"try" + 0.016*"seat" + 0.014*"mak

#### pyLDAvis: Interactive Topic Modelling Exploration

In [28]:
#!pip install pyLDAvis.gensim

In [29]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
pyLDAvis.enable_notebook()
visual_10 = pyLDAvis.gensim_models.prepare(lda_model_10, corpus, id2word)
visual_20 = pyLDAvis.gensim_models.prepare(lda_model_20, corpus, id2word)
visual_30 = pyLDAvis.gensim_models.prepare(lda_model_30, corpus, id2word)
visual_40 = pyLDAvis.gensim_models.prepare(lda_model_40, corpus, id2word)
visual_80 = pyLDAvis.gensim_models.prepare(lda_model_80, corpus, id2word)

c:\Users\adrie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\Users\adrie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\Users\adrie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\Users\adrie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_in

In [31]:
visual_10

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.025993 -0.022883       1        1  13.241364
6      0.012740 -0.012111       2        1  11.093719
3      0.004822  0.002590       3        1  10.382470
8     -0.010521 -0.003441       4        1  10.270417
2      0.030529  0.039102       5        1  10.102410
7     -0.006310 -0.005053       6        1   9.891135
0      0.012695  0.025445       7        1   9.218620
4      0.060642 -0.096012       8        1   8.790047
5      0.011210  0.099568       9        1   8.556969
1     -0.141800 -0.027206      10        1   8.452851, topic_info=      Term        Freq       Total Category  logprob  loglift
14    need   81.000000   81.000000  Default  30.0000  30.0000
84   check   77.000000   77.000000  Default  29.0000  29.0000
92    seat  104.000000  104.000000  Default  28.0000  28.0000
61      go   74.000000   74.000000  Default  27.0000  27.0000
113    say   71.000000   71.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
82     amp    5.664852   71.984454  Topic10  -5.2697  -0.0715
84   check    5.704401   77.896842  Topic10  -5.2627  -0.1435
28    make    5.704284   99.190413  Topic10  -5.2627  -0.3852
274   like    5.214669   44.272797  Topic10  -5.3525   0.3318
127  delay    5.407216  116.543363  Topic10  -5.3162  -0.5999

[697 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
720       3  0.119452     access
720       4  0.477806     access
720       5  0.119452     access
720       6  0.119452     access
720       7  0.119452     access
...     ...       ...        ...
503       2  0.155586  yesterday
503       3  0.077793  yesterday
503       4  0.155586  yesterday
503       7  0.155586  yesterday
503      10  0.311172  yesterday

[2289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 4, 9, 3, 8, 1, 5, 6, 2])

In [32]:
visual_20

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9     -0.025554 -0.004227       1        1  6.545404
8      0.009070 -0.015609       2        1  6.223023
15    -0.007207 -0.043052       3        1  6.156790
11     0.038888 -0.079713       4        1  5.620850
4     -0.004057 -0.029166       5        1  5.599848
6      0.028203 -0.018018       6        1  5.580210
17     0.007144  0.012870       7        1  5.552140
14     0.072282  0.042183       8        1  5.206980
3     -0.080187 -0.005160       9        1  5.062806
7      0.074284  0.060612      10        1  4.872343
19    -0.006817 -0.032136      11        1  4.824719
2     -0.025523 -0.006607      12        1  4.770480
13     0.016877  0.070851      13        1  4.736800
18     0.041867 -0.032314      14        1  4.698980
16    -0.025846  0.048905      15        1  4.638407
1     -0.067883 -0.102809      16        1  4.542982
5     -0.027568  0.055370      17        1  4.516272
12     0.042535 -0.084801      18        1  3.959789
10    -0.152295  0.084402      19        1  3.585474
0      0.091786  0.078418      20        1  3.305702, topic_info=      Term        Freq       Total Category  logprob  loglift
135   time  114.000000  114.000000  Default  30.0000  30.0000
127  delay  116.000000  116.000000  Default  29.0000  29.0000
278   work   48.000000   48.000000  Default  28.0000  28.0000
92    seat  104.000000  104.000000  Default  27.0000  27.0000
294   know   63.000000   63.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
113    say    3.125960   72.147328  Topic20  -4.9254   0.2706
58    help    3.125743   78.191664  Topic20  -4.9254   0.1900
159    bag    3.125614   90.717630  Topic20  -4.9255   0.0414
49    tell    2.401154   63.321378  Topic20  -5.1892   0.1372
243  every    2.101044   19.325862  Topic20  -5.3227   1.1905

[1334 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
177       4  0.273960  aadvantage
177       8  0.182640  aadvantage
177      12  0.091320  aadvantage
177      15  0.273960  aadvantage
177      17  0.091320  aadvantage
...     ...       ...         ...
125      16  0.041109         yes
125      17  0.041109         yes
125      18  0.123326         yes
125      19  0.041109         yes
125      20  0.041109         yes

[5053 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 16, 12, 5, 7, 18, 15, 4, 8, 20, 3, 14, 19, 17, 2, 6, 13, 11, 1])

In [33]:
visual_30

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3      0.025132  0.004420       1        1  4.538536
18    -0.014766  0.045633       2        1  4.188167
15    -0.067285 -0.039669       3        1  4.073750
27    -0.057508 -0.001333       4        1  3.964084
21     0.021173  0.043687       5        1  3.959682
24    -0.033766  0.075335       6        1  3.854800
16     0.019449  0.064875       7        1  3.772623
26    -0.055975 -0.027335       8        1  3.653909
10     0.016024  0.002557       9        1  3.519611
9      0.047011 -0.076367      10        1  3.496440
22    -0.039409  0.052450      11        1  3.464332
2      0.078613 -0.051596      12        1  3.386284
25    -0.020224 -0.024707      13        1  3.363255
19     0.072128 -0.034813      14        1  3.358653
4     -0.035496  0.035679      15        1  3.296078
20     0.076585 -0.001629      16        1  3.288520
0     -0.089648  0.024719      17        1  3.235133
6      0.012017  0.053869      18        1  3.227883
1      0.138199  0.007458      19        1  3.202394
17     0.008670  0.080585      20        1  3.158928
12    -0.022643 -0.094233      21        1  3.098406
14    -0.078370  0.004668      22        1  3.046849
23     0.052455  0.097628      23        1  2.978744
13    -0.024708 -0.095296      24        1  2.944825
29     0.029894  0.001670      25        1  2.925810
11     0.014308  0.044976      26        1  2.801372
5     -0.059633  0.039766      27        1  2.784191
7     -0.110870 -0.078634      28        1  2.708361
28     0.032370 -0.103366      29        1  2.505210
8      0.066273 -0.050996      30        1  2.203171, topic_info=       Term        Freq       Total Category  logprob  loglift
127   delay  117.000000  117.000000  Default  30.0000  30.0000
135    time  117.000000  117.000000  Default  29.0000  29.0000
91      one   63.000000   63.000000  Default  28.0000  28.0000
139   today   66.000000   66.000000  Default  27.0000  27.0000
82      amp   71.000000   71.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
12   change    1.938990   53.334086  Topic30  -4.9972   0.5009
84    check    1.938986   76.788499  Topic30  -4.9972   0.1364
301     hey    1.938973   40.628287  Topic30  -4.9972   0.7730
52     well    1.938962   35.352068  Topic30  -4.9972   0.9121
27     look    1.938951   30.639851  Topic30  -4.9972   1.0551

[2010 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
177       3  0.264681  aadvantage
177       4  0.088227  aadvantage
177      10  0.088227  aadvantage
177      11  0.088227  aadvantage
177      12  0.088227  aadvantage
...     ...       ...         ...
125      18  0.043015         yes
125      20  0.043015         yes
125      23  0.043015         yes
125      24  0.043015         yes
125      27  0.215074         yes

[6627 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 19, 16, 28, 22, 25, 17, 27, 11, 10, 23, 3, 26, 20, 5, 21, 1, 7, 2, 18, 13, 15, 24, 14, 30, 12, 6, 8, 29, 9])

In [34]:
visual_40

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
26     0.017409  0.013878       1        1  3.436625
33     0.012412 -0.026686       2        1  3.304762
34     0.007230  0.046499       3        1  3.219732
0     -0.007790  0.007896       4        1  3.168170
31     0.071785  0.017861       5        1  3.120700
32     0.028331 -0.034212       6        1  3.100076
39     0.035735  0.065387       7        1  3.062732
4     -0.000618  0.111219       8        1  3.037879
3      0.057111  0.074196       9        1  3.010260
18    -0.022643  0.070337      10        1  3.005163
17    -0.049584  0.001839      11        1  2.907709
16     0.047024 -0.038533      12        1  2.849407
11    -0.001864  0.056018      13        1  2.819449
13    -0.035354 -0.006502      14        1  2.720046
7      0.001081 -0.042075      15        1  2.683916
10     0.055939 -0.060424      16        1  2.662693
38    -0.027983  0.108794      17        1  2.609914
25     0.102394  0.029298      18        1  2.543938
37     0.009229 -0.070067      19        1  2.532352
21     0.040626 -0.071508      20        1  2.494601
8      0.046147 -0.047676      21        1  2.490124
22     0.012541 -0.052810      22        1  2.463012
20     0.049142 -0.002958      23        1  2.462392
15     0.035982 -0.025584      24        1  2.388327
6     -0.095581  0.003356      25        1  2.380415
30    -0.005462 -0.029307      26        1  2.332415
36     0.100576 -0.012891      27        1  2.301105
29    -0.000356 -0.024780      28        1  2.148566
28     0.048703  0.022770      29        1  2.113986
14    -0.045424 -0.081856      30        1  2.097465
24     0.018679  0.039092      31        1  2.077844
35    -0.021866 -0.092399      32        1  1.995392
9     -0.072511 -0.111405      33        1  1.990179
1      0.112072 -0.012812      34        1  1.985445
2     -0.096749  0.048885      35        1  1.891907
12    -0.131084  0.009452      36        1  1.867454
5     -0.091110  0.031437      37        1  1.785967
27    -0.018809  0.155388      38        1  1.752382
23    -0.061513 -0.036237      39        1  1.724545
19    -0.123845 -0.032879      40        1  1.460952, topic_info=         Term       Freq       Total Category  logprob  loglift
90        get  95.000000   95.000000  Default  30.0000  30.0000
124      take  71.000000   71.000000  Default  29.0000  29.0000
28       make  99.000000   99.000000  Default  28.0000  28.0000
126    cancel  67.000000   67.000000  Default  27.0000  27.0000
159       bag  90.000000   90.000000  Default  26.0000  26.0000
..        ...        ...         ...      ...      ...      ...
28       make   2.048948   99.580873  Topic40  -4.5312   0.3424
92       seat   1.112124  103.881818  Topic40  -5.1423  -0.3109
420      room   1.037210   10.275829  Topic40  -5.2120   1.9328
258  children   1.037185    7.692565  Topic40  -5.2120   2.2223
655       sit   1.037179   30.003240  Topic40  -5.2120   0.8613

[2526 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
177       3  0.183358  aadvantage
177      11  0.091679  aadvantage
177      12  0.091679  aadvantage
177      13  0.275037  aadvantage
177      16  0.091679  aadvantage
...     ...       ...         ...
526      26  0.046327         yet
526      29  0.185309         yet
526      34  0.046327         yet
526      37  0.046327         yet
526      38  0.046327         yet

[7769 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[27, 34, 35, 1, 32, 33, 40, 5, 4, 19, 18, 17, 12, 14, 8, 11, 39, 26, 38, 22, 9, 23, 21, 16, 7, 31, 37, 30, 29, 15, 25, 36, 10, 2, 3, 13, 6, 28, 24, 20])

In [35]:
visual_80

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
75    -0.049394 -0.028649       1        1  2.067263
56    -0.053131 -0.082209       2        1  1.924573
40    -0.072191 -0.067354       3        1  1.863240
37    -0.095733 -0.021944       4        1  1.740176
76     0.028315 -0.100837       5        1  1.669467
...         ...       ...     ...      ...       ...
67     0.051882  0.006853      76        1  0.862543
69     0.164913 -0.037176      77        1  0.753518
68     0.029222  0.046780      78        1  0.729838
0      0.017134  0.005553      79        1  0.705260
71     0.024890  0.060379      80        1  0.682893

[80 rows x 5 columns], topic_info=          Term        Freq       Total Category  logprob  loglift
127      delay  119.000000  119.000000  Default  30.0000  30.0000
28        make  100.000000  100.000000  Default  29.0000  29.0000
135       time  115.000000  115.000000  Default  28.0000  28.0000
129      hours   58.000000   58.000000  Default  27.0000  27.0000
124       take   69.000000   69.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
44         try    1.197671   57.291116  Topic80  -4.3077   1.1188
425  check_bag    1.197658   28.801155  Topic80  -4.3077   1.8065
282      staff    1.197657   31.544915  Topic80  -4.3077   1.7155
11        also    1.197653   21.448066  Topic80  -4.3077   2.1013
113        say    1.197650   75.272852  Topic80  -4.3077   0.8458

[4253 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
177       4  0.080544  aadvantage
177      10  0.080544  aadvantage
177      17  0.080544  aadvantage
177      31  0.161088  aadvantage
177      37  0.080544  aadvantage
...     ...       ...         ...
901      37  0.118585       young
901      46  0.118585       young
901      56  0.118585       young
901      62  0.118585       young
901      79  0.118585       young

[9472 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[76, 57, 41, 38, 77, 63, 37, 3, 8, 48, 58, 51, 43, 45, 40, 19, 26, 16, 30, 65, 78, 23, 31, 35, 39, 75, 15, 27, 2, 62, 32, 46, 17, 10, 9, 5, 67, 54, 11, 12, 59, 50, 42, 60, 28, 33, 73, 36, 53, 14, 80, 13, 56, 25, 7, 6, 74, 18, 71, 24, 4, 34, 21, 79, 66, 49, 64, 47, 61, 22, 52, 44, 29, 55, 20, 68, 70, 69, 1, 72])

The less it overlaps and the better is the distribution into topics

#### Model coherence

In [36]:
# lda model's coherence score
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda_10 = CoherenceModel(model=lda_model_10, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_20 = CoherenceModel(model=lda_model_20, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_30 = CoherenceModel(model=lda_model_30, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_40 = CoherenceModel(model=lda_model_40, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_80 = CoherenceModel(model=lda_model_80, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_lda_10 = coherence_model_lda_10.get_coherence()
coherence_lda_20 = coherence_model_lda_20.get_coherence()
coherence_lda_30 = coherence_model_lda_30.get_coherence()
coherence_lda_40 = coherence_model_lda_40.get_coherence()
coherence_lda_80 = coherence_model_lda_80.get_coherence()
print('Coherence Score_10: ', coherence_lda_10)
print('Coherence Score_20: ', coherence_lda_20)
print('Coherence Score_30: ', coherence_lda_30)
print('Coherence Score_40: ', coherence_lda_40)
print('Coherence Score_80: ', coherence_lda_80)

Coherence Score_10:  0.24611084181772536
Coherence Score_20:  0.30770951317840295
Coherence Score_30:  0.3434900865558782
Coherence Score_40:  0.3697981172793267
Coherence Score_80:  0.4233193062896493


more topics is more coherent but we must pick a good fit between coherence and a number of topics where classification is good enough

#### Try to extract topic from new text 

In [37]:
from operator import itemgetter

In [38]:
def preprocess(text):
    text_cleaned = gensim.utils.simple_preprocess(question_preprocess(text))
    text_lem = bigram_mod[lemmat(" ".join([w for w in text_cleaned if w not in stop_words]))]
    return text_lem

In [39]:
def get_highest_topic(text):
  text_preprocessed = preprocess(text)
  result = lda_model_30[id2word.doc2bow(text_preprocessed)]
  return max(result, key=itemgetter(1))[0]

get_highest_topic('@AmericanAir terrible service wait ages trying to call customer service almost two months gone no response')

22

In [40]:
data['topic'] = data['question'].apply(get_highest_topic)
data.head(10)

,responce,question,question_lem_bigrams,topic
0,@115904 We'll be sure to pass along your kind words! #AATeam,@AmericanAir Erica on the lax team is amazing give her a raise ty,"[erica, lax, team, amaze, give, raise]",4
1,@115904 Our apologies for the delay in responding to you. Have you made it to LAX? Let us know if you still need assistance.,@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP,"[could, someone, lax, team, available, guide, gate, asap]",8
2,"@115905 Aww, that's definitely a future pilot in the making! #HappyHalloween",Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1,"[ben, tennyson, american_airlines, pilot]",29
3,@115906 We're sorry for your frustration.,"@AmericanAir Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program.","[right, earn, also, pay, pass, spouse, need, change, program]",21
4,@115909 We're glad you got to kick back and enjoy a show while flying! Thanks for your kind words.,"Thank you, @AmericanAir for playing #ThisIsUs and for having great flight attendants on my flight back home!","[play, great, attendants, back, home]",2
5,@116142 We never want your experience to be anything less than perfect. We're sorry the Wi-Fi was slower than expected.,@AmericanAir's wifi makes Amtrak's wifi look pro. At least theirs is free. @5804 has decent wifi @ reasonable prices. Why not AA?,"[wifi, make, amtrak, wifi, look, pro, least, free, decent, wifi, reasonable, price]",0
6,"@116143 Thanks for the shout-out and for stopping by, Marc! We'll pass this on to our SFO Admirals Club managers.","Wonderful club! @americanair (@ American Airlines Admirals Club in San Francisco, CA) https://t.co/GWFwuGVgNA","[wonderful, club, american_airlines, admirals_club, san, francisco]",28
7,@116144 They'll be more than happy to walk you through it and help with the booking process.,"@AmericanAir already did...changed browsers, did all my techie tricks and still can't make a reservation after 1 hour!!","[already, change, browsers, techie, trick, still, make, reservation, hour]",24
8,@116144 We're sorry for the long wait. The next available agent will be with you as soon as possible.,@AmericanAir ........still....on....hold.....they made the booking. I am trying to pay.....so #frustrating.,"[still, hold, make, book, try, pay]",20
9,@116144 Fares sell in real time. We're sorry we weren't able to snag the lower fare for you.,@AmericanAir well now i am told the ticket cost is an additional $200/person to pay....because of the wait....omg,"[well, tell, ticket, cost, additional, person, pay, wait, omg]",14


#### Building Intents

In [41]:
#custom intents
intents = {"Greeting": ["Hello", "How are you doing?", "Greetings!", "How do you do?"],
          "Apology": ["No problem"],
          "Thanks": ["No problem", "You're welcome"],
          "Goodbye": ["It was nice speaking to you", "See you later", "Speak soon!"]
             }

In [42]:
def update_intents(df):
  for index, row in df.iterrows():
    if str(row['topic']) not in intents :
      intents[str(row['topic'])] = []
    if row['responce'] != "":
      intents[str(row['topic'])].append(str(row['responce']))

In [43]:
update_intents(data)

Save Intents in order to use it again

In [44]:
with open('intents.json', 'w') as fp:
    json.dump(intents, fp)

#### Doc2Vec

In [45]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [46]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(data['responce'])]
model = Doc2Vec(tagged_data, window=1, min_count=2, epochs = 200)

#### First Results 

In [47]:
def chatbot(text):
  result = model.dv.most_similar(positive=[model.infer_vector(preprocess(text))], topn=1850)
  topic = get_highest_topic(text)
  for l in result:
    if data['topic'].iloc[int(l[0])] == topic:
      return data['responce'].iloc[int(l[0])]

In [48]:
#Preexisting questions and responses testing

list_word = ["@AmericanAir what’s going on with flight 301 DTW &gt; DFW? Delayed over 3 hours. Will it be canceled?", 
             "@AmericanAir awful service, more than 1 hour delay due to logbook maintenance issues. Where is preventive maintenance?",
             "@AmericanAir Yes. Refund my plane ticket😡 with your racist ass workers",
             "@AmericanAir I'm aware. Fits in overhead.This was a return trip. Had same luggage going. This was pre-security. Gate chck, ok, but Empty overheads on flt",
             "Thank you @AmericanAir for destroying not one but BOTH pieces of luggage we checked with you. Paid a checked baggage fee so our luggage could be ruined. Never again."]

list_resp = ["Looks like it has a maintenance delay and it's currently scheduled to take off at 10:10p.",
             "We do many checks pre and post departure. Please share your flight number if we can provide an update.",
             "We don't tolerate discrimination of any kind. Please DM your record locator and contact details (phone and email)",
             "We'd like to share this feedback. Please DM your record locator.",
             "We're sorry about that, Tyler. Please take your bags to the Baggage Service Office to file a claim."]

for q,r in zip(list_word, list_resp):
  print("QUESTION :", q)
  print("EXPECTED :", r)
  print("PREDICTED :", chatbot(q),"\n")
  


QUESTION : @AmericanAir what’s going on with flight 301 DTW &gt; DFW? Delayed over 3 hours. Will it be canceled?
EXPECTED : Looks like it has a maintenance delay and it's currently scheduled to take off at 10:10p.
PREDICTED : @157794 We want to reunite you quickly with your bags. Please work directly with our Baggage team. 

QUESTION : @AmericanAir awful service, more than 1 hour delay due to logbook maintenance issues. Where is preventive maintenance?
EXPECTED : We do many checks pre and post departure. Please share your flight number if we can provide an update.
PREDICTED : @123413 We'll update your claim via this link: https://t.co/Lts67cwTlN 

QUESTION : @AmericanAir Yes. Refund my plane ticket😡 with your racist ass workers
EXPECTED : We don't tolerate discrimination of any kind. Please DM your record locator and contact details (phone and email)
PREDICTED : @137446 We aren't expecting any cancellations during the holiday season. If that changes though, we'll be sure to notify you,

In [49]:
question = input('Enter your question : ')
chatbot(question)

"@116418 We'd like to share this feedback. Please DM your record locator."